In [ ]:
! git clone https://github.com/yash-srivastava19/attention-free-revolution.git

Cloning into 'attention-free-revolution'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 82 (delta 27), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (82/82), 30.55 KiB | 665.00 KiB/s, done.
Resolving deltas: 100% (27/27), done.


In [ ]:
! pip install wandb

  Using cached wandb-0.16.2-py3-none-any.whl (2.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.5 MB/s eta 0:00:00


In [ ]:
import os
import math
import torch
import wandb
import warnings
import torch.nn as nn
import torch.nn.functional as F
from scipy.signal import correlate

In [ ]:
with open('data.txt', 'r', encoding='utf-8') as f:   ## add path to the dataset here.
  text = f.read()

chars = sorted(list(set(text)))  # possible elements from the dataset.
vocab_size = len(chars)

print(vocab_size)

# Tokenize the set(Character level tokenizer)
string_toInt = {ch: i for i, ch in enumerate(chars)}
int_toString = {i: ch for i, ch in enumerate(chars)}

# Forward and Reverse Mapping(String to Integer). There are different schemes, Google uses sentencepiece,
# OpenAI uses tiktoken
encode = lambda string: [string_toInt[c] for c in string]
decode = lambda lst: ''.join([int_toString[i] for i in lst])

data = torch.tensor(encode(text), dtype=torch.long)
N = int(len(data)*0.9)

print(N)

train_data = data[N:]
val_data = data[:N]

91
4912379


In [ ]:
""" Maybe we'll move this in the same file as well, or modify the function a lil bit(much more likely). """


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - training_config.block_size, (training_config.batch_size,))
    x = torch.stack([data[i:i+training_config.block_size] for i in ix])
    y = torch.stack([data[i+1:i+training_config.block_size+1] for i in ix])
    x, y = x.to(training_config.device), y.to(training_config.device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(training_config.eval_iters)
        for k in range(training_config.eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [ ]:
class BaseLeviathanConfig:
    attn_dropout = 0.1
    embed_dropout = 0.1
    ff_dropout = 0.1

    def __init__(self, vocab_size, max_len, **kwargs) -> None:
        self.vocab_size = vocab_size
        self.max_len = max_len

        for key, value in kwargs.items():
            setattr(self, key, value)


class LeviathanModelConfig(BaseLeviathanConfig):
    num_heads = 2
    num_blocks = 2
    embed_dim = 4

leviathan_model_config = LeviathanModelConfig(vocab_size, max_len=1000)     ## Todo: Change these or something like that.

class TrainingConfig:
    batch_size = 64         # how many independent sequences will we process in parallel?
    block_size = 512        # what is the maximum context length for predictions?
    max_iters = 10000        # How many epochs the model should be trained for?
    eval_interval = 100     # After how many intervals should we see the validation loss?
    ckpt_interval = 500     # After how many intervals should we save the model?
    learning_rate = 3e-4    # learning rate.
    eval_iters = 200
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

training_config = TrainingConfig()
print(training_config.device)

cuda


In [ ]:
class MultiHeadedCorrelation(nn.Module):
    def __init__(self, d_model, num_heads) -> None:
        super().__init__()
        assert d_model % num_heads == 0

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_correlation(self, Q, K, V, mask=None, dropout=None):
        """ [NEW: This is what is different from the scaled dot product attention.] """
        scores = torch.from_numpy(correlate(Q.detach().cpu().numpy(), K.detach().cpu().numpy(), mode='same'))
        scores = scores.to(training_config.device)

        scores = scores@K.transpose(-2, -1) / math.sqrt(self.d_k)

        if mask is not None:
            mask = mask.unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e9)

        scores = F.softmax(scores, dim=-1)

        if dropout is not None:
            scores = dropout(scores)

        output =  scores@V
        return output

    def _different_type_of_correlation(self, Q, K, V, mask=None, dropout=None):
        raise NotImplementedError


    def _another_different_type_of_correlation(self, Q, K, V, mask=None, dropout=None):
        raise NotImplementedError


    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)


    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self,x, mask=None):

        Q = self.split_heads(self.W_q(x))
        K = self.split_heads(self.W_k(x))
        V = self.split_heads(self.W_v(x))

        attn_output = self.scaled_correlation(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))

        return output

class Block(nn.Module):
    """ This is where we can allow for creativity. Just that we can go from embed_dim, and back to embed_dim """
    def __init__(self, config) -> None:
        super().__init__()
        embed_dim = config.embed_dim

        self.ln1 = nn.LayerNorm(embed_dim)
        self.ln2 = nn.LayerNorm(embed_dim)

        self.corr = MultiHeadedCorrelation(d_model = config.embed_dim, num_heads=config.num_heads)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim*4),
            nn.GELU(),
            nn.Linear(embed_dim*4, embed_dim),
            nn.Dropout(config.ff_dropout),
        )

    def forward(self, x):
        x = x + self.corr(self.ln1(x))
        x = x + self.ff(self.ln2(x))

        return x    # This is the x which goes to the decoder.

In [ ]:
class LeviathanComponentModel(nn.Module):
    def __init__(self, config) -> None:
        super().__init__()
        embed_dim = config.embed_dim
        self.max_len = config.max_len

        self.tok_embed = nn.Embedding(config.vocab_size, embed_dim)

        self.pos_embed = nn.Parameter(torch.zeros(1, config.max_len, embed_dim))

        self.dropout = nn.Dropout(config.embed_dropout)

        self.blocks = nn.Sequential(
            *[Block(config) for _ in range(config.num_blocks)]
        )

        self.ln = nn.LayerNorm(embed_dim)
        self.fc = nn.Linear(embed_dim, config.vocab_size)

    def forward(self, x, target=None):
        seq_len = x.size(1)

        assert seq_len <= self.max_len, f"Sequence Length longer than Model Capacity({seq_len}>{self.max_len})."

        tok_embed = self.tok_embed(x)   # toke_embed.shape = (batch_size, seq_len, embed_dim)
        pos_embed = self.pos_embed[:, :seq_len, :]  # pos_embed.shape = (1, seq_len, embed_dim)

        x = self.dropout(tok_embed + pos_embed)
        x = self.blocks(x)
        x = self.ln(x)
        logits = self.fc(x)   # x.shape = (batch_size, seq_len, vocab_size)

        if target is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)

        return logits, loss  # basically, token prediction per position.

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -training_config.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
class Tokenizer:
    """ Vanilla Tokenizer. Will train a BPE model and use that. """
    def __init__(self, text) -> None:
        self._text = text
        self._chars = sorted(list(set(self._text)))
        self._vocab_size = len(self._chars)

    def _string_to_integer(self):
        """ Utility function for creating a mapping from character to integers. Converts string to integer. """
        return {ch: i for i, ch in enumerate(self._chars)}


    def _integer_to_string(self):
        """ Utility function for creating a mapping from integers to character. Converts integer to string. """
        return {ch: i for i, ch in enumerate(self._chars)}


    def encode(self):
        """ Encode the text, and returns the list of integers. """
        warnings.warn('This is a pretty naive methos to create a mapping, and will not be scalable once large enough dataset is there. ', DeprecationWarning)
        return [self._string_to_integer(char) for char in self._text]


    def decode(self, lst_ints:list[int]):
        """ Decode the list of integers to convert them into string. Useful to see how the model understands things. """
        return ''.join([self._integer_to_string(i) for i in lst_ints])

In [ ]:
wandb.init()

step,▁▂▂▃▄▅▅▆▇▇█
train_loss,█▄▃▂▂▂▁▁▁▁▁
val_loss,█▄▃▂▂▂▁▁▁▁▁
step,4999
train_loss,2.86838
val_loss,2.87245


In [ ]:
""" Add the ability to track metrics and saving checkpoint, together with uploading them to HuggingFace. """


config = LeviathanModelConfig(vocab_size, max_len=1000)   ## Change this boi.
model = LeviathanComponentModel(config)

model = model.to(training_config.device)

print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

wandb.watch(model, log_freq = training_config.eval_interval)

optimizer = torch.optim.AdamW(model.parameters(), lr=training_config.learning_rate)

for iter in range(training_config.max_iters+1):
    #if iter % save_internval == 0
    if iter % training_config.eval_interval == 0 or iter == training_config.max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        wandb.log({'step': iter, 'train_loss': losses['train'], 'val_loss': losses['val']})

        # Uncomment to save the best models.

        #torch.save({
        #    'epoch': iter,
        #    'model_state_dict': model.state_dict(),
        #    'optimizer_state_dict': optimizer.state_dict(),
        #    'loss': losses,
        #    }, f'model_{iter}_{losses["train"]:.4f}')

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


0.005315 M parameters
step 0: train loss 4.7157, val loss 4.7154
step 100: train loss 4.4085, val loss 4.4033
step 200: train loss 4.1525, val loss 4.1437
step 300: train loss 3.9493, val loss 3.9378
step 400: train loss 3.7849, val loss 3.7732
step 500: train loss 3.6428, val loss 3.6279
step 600: train loss 3.5239, val loss 3.5072
step 700: train loss 3.4310, val loss 3.4116
step 800: train loss 3.3599, val loss 3.3443
step 900: train loss 3.3107, val loss 3.2895
step 1000: train loss 3.2772, val loss 3.2591
step 1100: train loss 3.2538, val loss 3.2394
step 1200: train loss 3.2344, val loss 3.2156
step 1300: train loss 3.2223, val loss 3.1985
step 1400: train loss 3.2090, val loss 3.1879
step 1500: train loss 3.1921, val loss 3.1742
step 1600: train loss 3.1768, val loss 3.1590
step 1700: train loss 3.1611, val loss 3.1417
step 1800: train loss 3.1418, val loss 3.1197
step 1900: train loss 3.1210, val loss 3.1026
step 2000: train loss 3.0995, val loss 3.0849
step 2100: train loss 3.